In [36]:
import numpy as np
import pandas as pd
import os
import warnings
import random
import torch 
from torch import nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
import tokenizers
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
import utils
from tqdm.autonotebook import tqdm

warnings.filterwarnings('ignore')

In [37]:
def seed_everything(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed = 43
seed_everything(seed)

In [3]:
import sentencepiece as spm
import sentencepiece_pb2
class SentencePieceTokenizer:
    def __init__(self, model_path):
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(os.path.join(model_path, "spiece.model"))
    
    def encode(self, sentence):
        spt = sentencepiece_pb2.SentencePieceText()
        spt.ParseFromString(self.sp.encode_as_serialized_proto(sentence))
        offsets = []
        tokens = []
        for piece in spt.pieces:
            tokens.append(piece.id)
            offsets.append((piece.begin, piece.end))
        return tokens, offsets

In [4]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, df, max_len=168):
        self.df = df
        self.max_len = max_len
        self.labeled = 'selected_text' in df
        self.tokenizer = SentencePieceTokenizer('../input/albert_base')

    def __getitem__(self, index):
        data = {}
        row = self.df.iloc[index]
        
        ids, masks, tweet, offsets = self.get_input_data(row)
        data['ids'] = ids
        data['masks'] = masks
        data['tweet'] = tweet
        data['offsets'] = offsets
        
        if self.labeled:
            start_idx, end_idx = self.get_target_idx(row, tweet, offsets)
            data['start_idx'] = start_idx
            data['end_idx'] = end_idx
        
        return data

    def __len__(self):
        return len(self.df)
    
    def get_input_data(self, row):
        tweet = " " + " ".join(row.text.lower().split())
        encoding = self.tokenizer.encode(tweet)
        sentiment_id = self.tokenizer.encode(row.sentiment)[0]
        ids = [0] + sentiment_id + [2, 2] + encoding[0] + [2]
        offsets = [(0, 0)] * 4 + encoding[1] + [(0, 0)]
                
        pad_len = self.max_len - len(ids)
        if pad_len > 0:
            ids += [1] * pad_len
            offsets += [(0, 0)] * pad_len
        
        ids = torch.tensor(ids)
        masks = torch.where(ids != 1, torch.tensor(1), torch.tensor(0))
        offsets = torch.tensor(offsets)
        
        return ids, masks, tweet, offsets
        
    def get_target_idx(self, row, tweet, offsets):
        selected_text = " " +  " ".join(row.selected_text.lower().split())

        len_st = len(selected_text) - 1
        idx0 = None
        idx1 = None

        for ind in (i for i, e in enumerate(tweet) if e == selected_text[1]):
            if " " + tweet[ind: ind+len_st] == selected_text:
                idx0 = ind
                idx1 = ind + len_st - 1
                break

        char_targets = [0] * len(tweet)
        if idx0 != None and idx1 != None:
            for ct in range(idx0, idx1 + 1):
                char_targets[ct] = 1

        target_idx = []
        for j, (offset1, offset2) in enumerate(offsets):
            if sum(char_targets[offset1: offset2]) > 0:
                target_idx.append(j)

        start_idx = target_idx[0]
        end_idx = target_idx[-1]
        
        return start_idx, end_idx
        
def get_train_val_loaders(df, train_idx, val_idx, batch_size=8):
    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]

    train_loader = torch.utils.data.DataLoader(
        TweetDataset(train_df), 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=8,
        drop_last=True)

    val_loader = torch.utils.data.DataLoader(
        TweetDataset(val_df), 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2)

    dataloaders_dict = {"train": train_loader, "val": val_loader}

    return dataloaders_dict

def get_test_loader(df, batch_size=32):
    loader = torch.utils.data.DataLoader(
        TweetDataset(df), 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2)    
    return loader

In [5]:
class TweetModel(nn.Module):
    def __init__(self):
        super(TweetModel, self).__init__()
        
        config = transformers.AlbertConfig.from_pretrained(
            '../input/albert_base/albert-base-v2-config.json', output_hidden_states=True)    
        self.roberta = transformers.AlbertModel.from_pretrained(
            '../input/albert_base/albert-base-v2-pytorch_model.bin', config=config)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(config.hidden_size, 2)
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)

    def forward(self, input_ids, attention_mask):
        _, _, hs = self.roberta(input_ids, attention_mask)
        
        x = torch.stack([hs[-1], hs[-2], hs[-3]])
        x = torch.mean(x, 0)[0]
        x = self.dropout(x)
        x = self.fc(x)
        start_logits, end_logits = x.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
                
        return start_logits, end_logits

In [6]:
def loss_fn(start_logits, end_logits, start_positions, end_positions):
    ce_loss = nn.CrossEntropyLoss()
    start_loss = ce_loss(start_logits, start_positions)
    end_loss = ce_loss(end_logits, end_positions)    
    total_loss = start_loss + end_loss
    return total_loss

In [7]:
def get_selected_text(text, start_idx, end_idx, offsets):
    selected_text = ""
    for ix in range(start_idx, end_idx + 1):
        selected_text += text[offsets[ix][0]: offsets[ix][1]]
        if (ix + 1) < len(offsets) and offsets[ix][1] < offsets[ix + 1][0]:
            selected_text += " "
    return selected_text

def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def compute_jaccard_score(text, start_idx, end_idx, start_logits, end_logits, offsets):
    start_pred = np.argmax(start_logits)
    end_pred = np.argmax(end_logits)
    if start_pred > end_pred:
        pred = text
    else:
        pred = get_selected_text(text, start_pred, end_pred, offsets)
        
    true = get_selected_text(text, start_idx, end_idx, offsets)
    
    return jaccard(true, pred)

In [8]:
def train_model(model, dataloaders_dict, criterion, optimizer, scheduler, num_epochs, filename):
    model.cuda()

    for epoch in range(num_epochs):
        losses = utils.AverageMeter()
        jaccards = utils.AverageMeter()
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
                
            tk0 = tqdm(dataloaders_dict[phase], total=len(dataloaders_dict[phase]))
            epoch_loss = 0.0
            epoch_jaccard = 0.0
            
            #for data in (dataloaders_dict[phase]):
            for bi, data in enumerate(tk0):
                ids = data['ids'].cuda()
                masks = data['masks'].cuda()
                tweet = data['tweet']
                offsets = data['offsets'].numpy()
                start_idx = data['start_idx'].cuda()
                end_idx = data['end_idx'].cuda()

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):

                    start_logits, end_logits = model(ids, masks)

                    loss = criterion(start_logits, end_logits, start_idx, end_idx)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()

                    epoch_loss += loss.item() * len(ids)
                    
                    start_idx = start_idx.cpu().detach().numpy()
                    end_idx = end_idx.cpu().detach().numpy()
                    start_logits = torch.softmax(start_logits, dim=1).cpu().detach().numpy()
                    end_logits = torch.softmax(end_logits, dim=1).cpu().detach().numpy()
                    
                    jaccard_scores = []
                    for i in range(len(ids)):                        
                        jaccard_score = compute_jaccard_score(
                            tweet[i],
                            start_idx[i],
                            end_idx[i],
                            start_logits[i], 
                            end_logits[i], 
                            offsets[i])
                        epoch_jaccard += jaccard_score
                        jaccard_scores.append(jaccard_score)
                    jaccards.update(np.mean(jaccard_scores), len(ids))
                    losses.update(loss.item(), len(ids))
                    tk0.set_postfix(loss=losses.avg, jaccard=jaccards.avg)
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_jaccard = epoch_jaccard / len(dataloaders_dict[phase].dataset)
            
            print('Epoch {}/{} | {:^5} | Loss: {:.4f} | Jaccard: {:.4f}'.format(
                epoch + 1, num_epochs, phase, epoch_loss, epoch_jaccard))
    
    torch.save(model.state_dict(), filename)

In [9]:
num_epochs = 3
batch_size = 32
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

In [10]:
%%time

train_df = pd.read_csv('../input/tweet-sentiment-extraction/train.csv')
train_df['text'] = train_df['text'].astype(str)
train_df['selected_text'] = train_df['selected_text'].astype(str)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df.sentiment), start=1): 
    print(f'Fold: {fold}')

    model = TweetModel()
    num_training_steps = int(len(train_df) / batch_size * num_epochs)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5, eps =1e-8, betas=(0.9, 0.999))
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=814, num_training_steps=num_training_steps)
    criterion = loss_fn    
    dataloaders_dict = get_train_val_loaders(train_df, train_idx, val_idx, batch_size)

    train_model(
        model, 
        dataloaders_dict,
        criterion,
        optimizer, 
        scheduler,
        num_epochs,
        f'albert_finetune_fold{fold}.pth')

Fold: 1



Epoch 1/3 | train | Loss: 5.6468 | Jaccard: 0.5011



Epoch 1/3 |  val  | Loss: 2.2033 | Jaccard: 0.6828



Epoch 2/3 | train | Loss: 1.9007 | Jaccard: 0.6898



Epoch 2/3 |  val  | Loss: 1.8171 | Jaccard: 0.6917


...


Epoch 1/3 |  val  | Loss: 2.1755 | Jaccard: 0.6591



Epoch 2/3 | train | Loss: 1.9098 | Jaccard: 0.6917



Epoch 2/3 |  val  | Loss: 1.8237 | Jaccard: 0.6928



Epoch 3/3 | train | Loss: 1.5629 | Jaccard: 0.7315



Epoch 3/3 |  val  | Loss: 1.7880 | Jaccard: 0.6991
CPU times: user 1h 2min 30s, sys: 6min 20s, total: 1h 8min 50s
Wall time: 1h 8min 52s


In [11]:
%%time

test_df = pd.read_csv('../input/tweet-sentiment-extraction/test.csv')
test_df['text'] = test_df['text'].astype(str)
test_loader = get_test_loader(test_df)
predictions = []
models = []
for fold in range(skf.n_splits):
    model = TweetModel()
    model.cuda()
    model.load_state_dict(torch.load(f'albert_fold{fold+1}.pth'))
    model.eval()
    models.append(model)

for data in test_loader:
    ids = data['ids'].cuda()
    masks = data['masks'].cuda()
    tweet = data['tweet']
    offsets = data['offsets'].numpy()

    start_logits = []
    end_logits = []
    for model in models:
        with torch.no_grad():
            output = model(ids, masks)
            start_logits.append(torch.softmax(output[0], dim=1).cpu().detach().numpy())
            end_logits.append(torch.softmax(output[1], dim=1).cpu().detach().numpy())

    start_logits = np.mean(start_logits, axis=0)
    end_logits = np.mean(end_logits, axis=0)
    for i in range(len(ids)):    
        start_pred = np.argmax(start_logits[i])
        end_pred = np.argmax(end_logits[i])
        if start_pred > end_pred:
            pred = tweet[i]
        else:
            pred = get_selected_text(tweet[i], start_pred, end_pred, offsets[i])
        predictions.append(pred)

FileNotFoundError: [Errno 2] No such file or directory: 'albert_fold1.pth'

In [12]:
sub_df = pd.read_csv('../input/tweet-sentiment-extraction/sample_submission.csv')
sub_df['selected_text'] = predictions
sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('!!!!', '!') if len(x.split())==1 else x)
sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('..', '.') if len(x.split())==1 else x)
sub_df['selected_text'] = sub_df['selected_text'].apply(lambda x: x.replace('...', '.') if len(x.split())==1 else x)
sub_df.to_csv('submission.csv', index=False)
sub_df.head()

ValueError: Length of values does not match length of index

In [ ]:
import numpy as np
import pandas as pd
train_df = pd.read_csv('../input/tweet-sentiment-extraction/train.csv').fillna("")
for i in train_df.index:
    text2 = train_df.loc[i, 'selected_text']
    text1 = train_df.loc[i, 'text']
    j = text1.find(text2)
    if j < 0:
        print(text1, text2)
        break

In [62]:
seed_everything(seed)
tokenizer = AlbertTokenizer.from_pretrained('../input/albert_base')
model = AlbertForQuestionAnswering.from_pretrained('../input/albert_base')
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_text = question + " [SEP] " + text
input_ids = tokenizer.encode(input_text)
token_type_ids = [0 if i <= input_ids.index(3) else 1 for i in range(len(input_ids))] # for albert [SEP] token has id 3
print(torch.tensor([input_ids]),torch.tensor([token_type_ids]))
start_scores, end_scores = model(torch.tensor([input_ids]), 
                                 token_type_ids=torch.tensor([token_type_ids]),
                                 attention_mask=torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))
print(start_scores, end_scores)
all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
print(all_tokens)
print([tokenizer.convert_ids_to_tokens(input_id) for input_id in input_ids])
print(' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))

tensor([[    2,    72,    23,  2170, 27674,    60,     3,  2170, 27674,    23,
            21,  2210, 10956,     3]]) tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]])
tensor([[ 0.3391,  0.7766, -0.5077,  0.6104, -0.1059, -0.2076,  0.5010,  0.5462,
          0.0228, -0.3135, -0.1714, -0.6611, -0.1446,  0.5011]],
       grad_fn=<SqueezeBackward1>) tensor([[ 0.0105,  0.1345,  0.5118,  0.7547, -0.6784,  0.0130,  0.0347,  0.4349,
         -0.7006,  0.2000, -0.6542, -0.2448, -0.3001,  0.0347]],
       grad_fn=<SqueezeBackward1>)
['[CLS]', '▁who', '▁was', '▁jim', '▁henson', '?', '[SEP]', '▁jim', '▁henson', '▁was', '▁a', '▁nice', '▁puppet', '[SEP]']
['[CLS]', '▁who', '▁was', '▁jim', '▁henson', '?', '[SEP]', '▁jim', '▁henson', '▁was', '▁a', '▁nice', '▁puppet', '[SEP]']
▁who ▁was ▁jim


In [57]:
from transformers import AlbertTokenizer, AlbertForQuestionAnswering
import torch
seed_everything(seed)

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForQuestionAnswering.from_pretrained('albert-base-v2')
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_dict = tokenizer.encode_plus(question, text, return_tensors='pt')
start_scores, end_scores = model(**input_dict)
print(input_dict)
print(start_scores, end_scores)
print(torch.argmax(start_scores),torch.argmax(end_scores)+1)
all_tokens = tokenizer.convert_ids_to_tokens(input_dict['input_ids'][0])
print(all_tokens)
print(' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))

{'input_ids': tensor([[    2,    72,    23,  2170, 27674,    60,     3,  2170, 27674,    23,
            21,  2210, 10956,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[ 0.3391,  0.7766, -0.5077,  0.6104, -0.1059, -0.2076,  0.5010,  0.5462,
          0.0228, -0.3135, -0.1714, -0.6611, -0.1446,  0.5011]],
       grad_fn=<SqueezeBackward1>) tensor([[ 0.0105,  0.1345,  0.5118,  0.7547, -0.6784,  0.0130,  0.0347,  0.4349,
         -0.7006,  0.2000, -0.6542, -0.2448, -0.3001,  0.0347]],
       grad_fn=<SqueezeBackward1>)
tensor(1, grad_fn=<NotImplemented>) tensor(4, grad_fn=<AddBackward0>)
['[CLS]', '▁who', '▁was', '▁jim', '▁henson', '?', '[SEP]', '▁jim', '▁henson', '▁was', '▁a', '▁nice', '▁puppet', '[SEP]']
▁who ▁was ▁jim


In [70]:
print(tokenizer.convert_tokens_to_ids("[SEP]"))

3


In [11]:
import transformers
import torch
AlbertTokenizer = transformers.AlbertTokenizer.from_pretrained('../input/albert_base')
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = transformers.AlbertForQuestionAnswering.from_pretrained('albert-base-v2')
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
input_ids = tokenizer.encode(input_text)
token_type_ids = [0 if i <= input_ids.index(3) else 1 for i in range(len(input_ids))] 
start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
all_tokens = tokenizer.convert_ids_to_tokens(input_ids)  
print(' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))
# a nice puppet

▁was ▁jim ▁henson ? [SEP] ▁jim ▁henson ▁was ▁a


In [13]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = transformers.AlbertForSequenceClassification.from_pretrained('albert-base-v2')
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute")).unsqueeze(0)  # Batch size 1
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(input_ids, labels=labels)
loss, logits = outputs[:2]

In [14]:
loss, logits

(tensor(1.0880, grad_fn=<NllLossBackward>),
 tensor([[ 0.2503, -0.4269]], grad_fn=<AddmmBackward>))